In [1]:
# Scraping wiki tables from this list
# https://ru.wikipedia.org/wiki/Список_стран_по_ВВП
from IPython.display import display
import requests
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
gdp_dict = {
    "ВВП Номинал": {
        'url': "https://ru.wikipedia.org/wiki/Список_стран_по_ВВП_(номинал)", 
        'table_position_wiki': 1,  # if there are multiple tables in the wiki html
        'columns_slice': [1, 3]},  # what column (usually what bank or what year) you want to pick
    "ВВП Номинал на человека": {
        'url': "https://ru.wikipedia.org/wiki/Список_стран_по_ВВП_(номинал)_на_душу_населения", 
        'table_position_wiki': 0,
        'columns_slice': [1, 2]},
    "ВВП ППС": {
        'url': "https://ru.wikipedia.org/wiki/Список_стран_по_ВВП_(ППС)", 
        'table_position_wiki': 0,
        'columns_slice': [1, 3]},
    "ВВП ППС на человека": {
        'url': "https://ru.wikipedia.org/wiki/Список_стран_по_ВВП_(ППС)_на_душу_населения", 
        'table_position_wiki': 0,
        'columns_slice': [1, 3]},
}
def explore_wiki_tables(url: str):
    tables = scrape_wiki(url)
    for _ in tables:
        display(pd.read_html(str(_))[0])
        
# scraping block
def scrape_wiki(url: str):
    table_class = "wikitable sortable jquery-tablesorter"
    response = requests.get(url)

    # parse data from the html into a beautifulsoup object
    soup = BeautifulSoup(response.text, 'html.parser')
    s = soup.find_all('table',{'class':"wikitable"})
    return s

scrape_results = {}
for k, v in gdp_dict.items():
    scrape_results[k] = scrape_wiki(v['url'])

    
# data processing block
def make_int(val):
    if isinstance(val, str):
        val = val.replace('\xa0', '').replace(' ', '')
    try:
        return int(val)
    except Exception as e:
        return np.NaN
    
resulting_dfs = []
for k, v in gdp_dict.items():
    html_table = scrape_results[k][v['table_position_wiki']]
    df = pd.read_html(str(html_table))
    df = pd.DataFrame(df[0])
    df = df.iloc[:, v['columns_slice']]
    df.iloc[:,1] = df.iloc[:,1].apply(make_int)  
    df.columns = ['Страна', k]
    resulting_dfs.append(df)
# write data to an Excel file
writer = pd.ExcelWriter('ВВП_4.xlsx', engine='openpyxl')
for i, _ in enumerate(gdp_dict.keys()):
    resulting_dfs[i].to_excel(writer, sheet_name=_, index=False)

info_page = pd.DataFrame([[k, v['url']] for k, v in gdp_dict.items()], columns=['Показатель', 'Ссылка'])
info_page.to_excel(writer, sheet_name='INFO', index=False)
writer.save()

In [2]:
wiki_item = gdp_dict['ВВП ППС']
u = wiki_item['url']
print(u)
explore_wiki_tables(u)

https://ru.wikipedia.org/wiki/Список_стран_по_ВВП_(ППС)


№      Страна (экономика) Данные ВБ[1][2]           Данные МВФ[3]  \
      №      Страна (экономика)           20200     20210         20200   
0   NaN                   Китай         24256.0   27313.0       24168.0   
1   NaN                     США         20894.0   22996.0       20894.0   
2   NaN                   Индия          9005.0   10219.0        9005.0   
3   NaN                  Япония          5316.0    5397.0        5295.0   
4   NaN                Германия          4561.0    4815.0        4573.0   
..   ..                     ...             ...       ...           ...   
199 NaN  Теркс и Кайкос (Брит.)            86.0       2.0           NaN   
200 NaN                   Земля        133745.0  146736.0      132487.0   
201 NaN                    ОЭСР         61528.0   66703.0           NaN   
202 NaN        Европейский союз         20055.0   21648.0       19781.0   
203 NaN                Еврозона         16158.0   17411.0           NaN   

              Доля в мире,2021, в %[4]  
        20210 Доля в мире,2021, в %[4]  
0     27206.0                     1856  
1     22996.0                     1569  
2     10194.0                      695  
3      5607.0                      382  
4      4888.0                      333  
..        ...                      ...  
199       NaN                      NaN  
200  146124.0                    10000  
201       NaN                      NaN  
202   21706.0                     1485  
203       NaN                      NaN  

[204 rows x 7 columns]

Страна (экономика)      ВВП (ППС) за 2019 год, млрд USD  \
   Страна (экономика) по данным МВФ за апрель 2020 года[6]   
0             Украина                                 4098   
1            Киргизия                                  261   
2            Молдавия                                  273   
3             Армения                                  334   
4              Грузия                                  479   
5            Ирландия                                 4179   
6             Франция                                30618   
7            Германия                                44436   
8                Ирак                                 7083   
9                Иран                                14914   
10              Китай                               273070   
11              Индия                               110432   
12          Индонезия                                37356   
13             Россия                                43900   

                                         Изменение, %  
   по данным МВФ за октябрь 2020 года[7] Изменение, %  
0                                   5610      +36,9 %  
1                                    352      +34,9 %  
2                                    360      +31,9 %  
3                                    421      +26,0 %  
4                                    582      +21,5 %  
5                                   4552       +8,9 %  
6                                  32280       +5,4 %  
7                                  46720       +5,1 %  
8                                   4479      −36,8 %  
9                                  10448      −29,9 %  
10                                233930      −14,3 %  
11                                 95423      −13,6 %  
12                                 33319      −10,8 %  
13                                 41360       −5,8 %